In [ ]:
import numpy as np 
import pandas as pd 
import re
import tqdm
import spacy

from collections import Counter
from tqdm import tqdm

pd.set_option('display.max_rows', 1000)
pd.set_option('max_colwidth', 800)

In [ ]:
def text_transform(text):
    text = text.lower()
    text = re.sub(r'-', r' - ', text)
    text = re.sub(r'\.', r' . ', text)
    text = re.sub(r'\?', r' ? ', text)
    text = re.sub(r'\!', r' ! ', text)
    text = re.sub(r'\,', r' , ', text)
    text = re.sub(r'\(', r' ( ', text)
    text = re.sub(r'\)', r' ) ', text)
    text = re.sub(r'\{', r' { ', text)
    text = re.sub(r'\}', r' } ', text)
    text = re.sub(r'\[', r' [ ', text)
    text = re.sub(r'\]', r' ] ', text)
    text = re.sub(r'\:', r' : ', text)
    text = re.sub(r'\;', r' ; ', text)
    text = re.sub(r'\"', r' " ', text)
    text = re.sub(r"\'", r" ' ", text)
    text = re.sub(r'\+', r' + ', text)
    text = re.sub(r'\/', r' / ', text)
    text = re.sub(r'\\', r' / ', text)
    text = re.sub(r'\&', r' & ', text)
    text = re.sub(r'\_', r' _ ', text)
    text = re.sub(r'\=', r' = ', text)
    text = re.sub(r'\*', r' * ', text)
    text = re.sub(r'\<', r' < ', text)
    text = re.sub(r'\>', r' > ', text)
    text = re.sub(r'\%', r' % ', text)
    text = re.sub(r'\~', r' ~ ', text)
    text = re.sub(r'(?P<num>\d+)', r' NUM ', text)
    return text

def word_counter(texts):
    words = []
    for index, row in texts.iterrows():
        annotation = eval(row['annotation'])
        for text in annotation:
            text = text_transform(text)
            text = text.split()
            words = words + text
    return sorted(Counter(words).items())

In [ ]:
train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
words = pd.read_csv('../input/score-clinical-patient/words.csv')

In [ ]:
numbers = ['NUM', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'few']
age = [('yo', 520), ('year', 206), ('old', 190), ('y', 147), ('o', 132), ('yr', 17), ('f', 4), ('years', 3), ('olf', 2), ('ye', 1), ('yeo', 1), ('odl', 1), 'age', 'ag']
male_or_female = [('f', 394), ('female', 248), ('m', 156), ('male', 123), ('ms', 41), ('woman', 35), ('mr', 17), ('mrs', 13), ('man', 6), ('yo', 4), ('g', 3), ('p', 3), ('lady', 2), ('aaf', 2), ('boy', 1), ('yof', 1), ('wf', 1), ('gentleman', 1), ('females', 1), ('femalwe', 1), ('women', 1)]
family_or_history = [('mother', 158), ('son', 99), ('uncle', 77), ('mom', 71), ('ulcer', 71), ('father', 67), ('fh', 43), ('hx', 17), ('family', 16), ('history', 14), ('fmhx', 3), ('fam', 3), ('fathers', 2), ('ulcers', 2), ('child', 2), ('famhx', 2), ('brother', 1), ('unlce', 1), ('parental', 1), ('ucler', 1), ('unkle', 1), ('uncles', 1), ('mum', 1), ('mpther', 1), ('fm', 1), ('pfh', 1), ('fmh', 1), 'dad']
heart = [('heart', 179), ('cardiac', 2), ('herat', 1), ('heaart', 1)]
attack = [('attack', 28), ('attcak', 1), ('attach', 1)]
mi_or_myocardial_infarction = [('mi', 53), ('myocardial', 1), ('infarction', 1)]
thyroid = [('thyroid', 84), ('thryoid', 2), ('hypothyroidism', 2), ('hyperthyroid', 2), ('hyperthyroidism', 1), ('thyoid', 1), ('hyroid', 1)]
disorder_or_disease_or_problem = [('diease', 1), ('disease', 26), ('disorder', 5), ('problem', 31), ('problems', 10)]
chest = [('chest', 237), ('cp', 9), ('midsternal', 2), ('substernal', 2), ('center', 2), ('middle', 1), ('ches', 1)]
pressure = [('pressure', 62), ('presure', 1)]
intermittent = [('intermittent', 42), ('intermittant', 2), ('intermittently', 2), ('intemittent', 2), ('intermiitent', 1), ('intermittnetly', 1), ('intermitent', 1)]
symptoms = [('symptoms', 27), ('symtpoms', 1)]
lightheaded = [('headed', 4), ('headedness', 10), ('lighheadeness', 1), ('lightheaded', 9), ('lightheadedness', 15), ('lightheadness', 2)]
no = [('deneis', 1), ('denes', 1), ('denied', 4), ('denies', 525), ('deniess', 1), ('deny', 2), ('didn', 3), ('didnt', 1), ('doesn', 1), ('doesnt', 3), ('don', 1), ('dont', 2), ('n', 2), ('neg', 2), ('negates', 1), ('negative', 16), ('neither', 1), ('never', 2), ('nil', 1), ('no', 388), ('non', 12), ('none', 1), ('not', 116), ('nothing', 1), ('nv', 1)]
hair_or_nail_or_skin = [('hair', 28), ('hairloss', 1), ('nail', 2), ('nails', 2), ('skin', 20)]
temperature = [('temperature', 6), ('temperaturre', 1)]
intolerance = [('inotlerance', 1), ('intolearance', 1), ('intolencace', 1), ('intolerance', 52), ('intolerenace', 1), ('intorelance', 1)]
use = [('use', 84), ('used', 13), ('uses', 15), ('using', 9)]
adderall = [('adderal', 23), ('adderall', 62), ('adderol', 4), ('adderoll', 3), ('adderral', 6), ('adderrall', 6), ('adderrol', 2), ('addirall', 2), ('addral', 1), ('addrall', 1), ('addreall', 1), ('addrell', 2), ('aderal', 1), ('aderall', 6), ('aderol', 6), ('aderral', 3), ('aderrall', 1), ('aderrol', 1), ('aldereol', 1), ('alderol', 1), ('amphetamine', 1), ('amphetimine', 1)]
breath = [('brathing', 1), ('breath', 96), ('breathe', 1), ('breathing', 35), ('breathlessness', 1), ('breaths', 16), ('breething', 1), ('brreathing', 1), ('btreath', 1)]
sob = [('sob', 129), ('soib', 1)]
short = [('shorntess', 1), ('short', 11), ('shorten', 1), ('shortness', 58)]
difficulty_or_trouble = [('difficullty', 2), ('difficult', 1), ('difficulties', 1), ('difficultly', 1), ('difficults', 1), ('difficulty', 156), ('difficulyt', 2), ('difficuty', 2), ('diffuclty', 2), ('diffuculuties', 1), ('diufficulty', 1), ('failing', 1), ('hard', 1), ('ifficulty', 1), ('toruble', 1), ('trouble', 100), ('troubles', 1), ('trouvble', 1), ('truble', 1), ('unable', 4)]
caffeine = [('bull', 2), ('bulls', 2), ('caffe', 1), ('caffee', 1), ('caffeiene', 1), ('caffeinated', 1), ('caffeine', 23), ('caffeinge', 1), ('caffenated', 1), ('caffiene', 3), ('caffine', 3), ('cofee', 1), ('coffee', 41), ('coffeine', 1), ('coffie', 2), ('cup', 3), ('cupps', 1), ('cups', 18), ('red', 4), ('redbull', 2)]
pounding_or_racing = [('bearting', 1), ('beat', 3), ('beating', 8), ('heartbeat', 1), ('jump', 4), ('jumping', 2), ('poudning', 1), ('pound', 1), ('pounding', 75), ('pounds', 1), ('punding', 2), ('race', 1), ('racing', 43), ('rapid', 1), ('rate', 2)]
tachycardia = [('tachycardia', 2)]
tremor = [('tremor', 4), ('tremors', 13)]
vaginal = [('v', 1), ('vagina', 2), ('vaginal', 79), ('vagnal', 2)]
discharge = [('discharge', 23)]
weight = [('ibs', 1), ('lb', 7), ('lbs', 22), ('pound', 1), ('pounds', 16), ('punds', 1), ('weght', 1), ('weight', 120), ('weighty', 1), ('wieght', 1), ('wight', 1), ('wt', 10)]
increase_or_gain = [('gain', 62), ('gained', 26), ('gaining', 6), ('inc', 1), ('incr', 1), ('increaed', 1), ('increase', 13), ('increased', 74), ('increases', 1), ('increasing', 7), ('incrrease', 1), ('intensified', 1)]
decreased_or_loss = [('dec', 6), ('decreaed', 1), ('decrease', 24), ('decreased', 97), ('decreasing', 1), ('decresed', 1), ('diminished', 2), ('down', 2), ('drain', 1), ('drained', 10), ('lack', 9), ('less', 7), ('looosely', 1), ('loose', 4), ('loosely', 1), ('loosening', 2), ('looser', 10), ('loosing', 1), ('los', 1), ('lose', 3), ('loser', 1), ('loses', 1), ('losing', 2), ('loss', 99), ('lost', 24), ('reduced', 7)]
energy = [('energry', 1), ('energy', 47), ('neergy', 1)]
sex = [('birth', 1), ('encounter', 8), ('intercourse', 15), ('intercouse', 1), ('monogamous', 8), ('monogomous', 3), ('oral', 1), ('partner', 4), ('partners', 1), ('seually', 1), ('seuxual', 1), ('sex', 25), ('sexual', 34), ('sexually', 106), ('sexualy', 2)]
protected_or_unprotected = [('barrier', 4), ('condom', 12), ('condomns', 1), ('condoms', 20), ('contraception', 19), ('contraceptive', 4), ('contraceptives', 4), ('contracepton', 1), ('control', 1), ('protection', 7), ('unprotected', 3)]
active = [('acitve', 1), ('active', 113), ('activity', 8), ('actvity', 1)]
diarrhea = [('diahrrea', 1), ('diarhea', 1), ('diarrhea', 157), ('diarrhoe', 2), ('diarrhoea', 4)]
bowel = [('bowel', 28)]
time = [('april', 1), ('aug', 2), ('august', 2), ('current', 1), ('currently', 5), ('d', 24), ('daily', 12), ('dates', 1), ('day', 134), ('days', 209), ('duration', 17), ('during', 5), ('ever', 2), ('everyday', 1), ('february', 2), ('hour', 3), ('hours', 74), ('hr', 3), ('hrs', 20), ('interval', 3), ('intervals', 2), ('july', 2), ('jun', 1), ('june', 2), ('min', 9), ('mins', 6), ('mintues', 1), ('minutes', 27), ('minuties', 1), ('mnths', 1), ('mo', 42), ('mod', 1), ('momths', 1), ('mongs', 1), ('monsths', 1), ('month', 41), ('months', 499), ('montly', 1), ('monts', 1), ('mos', 11), ('most', 4), ('motnhs', 2), ('motnsh', 1), ('nmonths', 1), ('november', 1), ('now', 17), ('oct', 1), ('october', 2), ('once', 6), ('ongoing', 1), ('time', 16), ('timeframe', 1), ('times', 41), ('timing', 1), ('today', 1), ('twice', 10), ('week', 80), ('weekend', 1), ('weekends', 1), ('weekly', 2), ('weeks', 371), ('weels', 1), ('wk', 15), ('wks', 34), ('wnl', 14), ('y', 7), ('year', 43), ('years', 180), ('yeasterday', 1), ('yeras', 1), ('yesreday', 1), ('yesterday', 139), ('yeswterday', 1), ('yeterday', 3), ('yo', 1), ('yr', 8), ('yrs', 17)]
past = [('ago', 595), ('asting', 1), ('away', 31), ('back', 9), ('before', 22), ('las', 1), ('last', 278), ('lasted', 8), ('lasting', 33), ('lastrs', 1), ('lasts', 17), ('lating', 1), ('passed', 27), ('passing', 3), ('passsed', 1), ('past', 174), ('previosly', 1), ('previous', 10), ('previously', 44), ('prior', 20), ('recent', 15), ('recently', 24)]
episodes = [('episdoes', 1), ('episode', 7), ('episodes', 78), ('episodic', 1), ('peisodes', 1)]
blood_or_bleeding = [('black', 2), ('bledding', 1), ('bleedign', 1), ('bleeding', 57), ('blood', 72), ('bloody', 12), ('blooody', 1), ('bright', 5), ('hematochezia', 3), ('hematoschezia', 1), ('hemochezia', 1), ('hemotochezia', 1), ('melena', 5), ('nonbloody', 5), ('nonbloudy', 1), ('red', 6)]
pain = [('discomfort', 2), ('hurts', 1), ('knife', 2), ('pain', 417), ('sharp', 79), ('stabbing', 15), ('stabby', 1), ('stiff', 1), ('stiffness', 16)]
acute = [('acute', 2)]
rlp_or_rlq = [('lrq', 3), ('rlq', 64)]
lower = [('low', 2), ('lowe', 1), ('lower', 34)]
right = [('r', 1), ('right', 33)]
abdominal = [('abd', 2), ('abdomen', 2), ('abdominal', 42), ('abdomonal', 1), ('abominla', 1), ('badominal', 1)]
urinary = [('disura', 1), ('dysuria', 9), ('hematuria', 3), ('micturition', 1), ('urgency', 2), ('urinary', 27), ('urination', 12), ('urine', 1)]
changes = [('chagnes', 1), ('chan', 1), ('change', 25), ('changes', 96)]
habits = [('habit', 2), ('habits', 15)]
appetite = [('anorexia', 7), ('apetite', 13), ('apettite', 1), ('appeitite', 2), ('appertite', 1), ('appetie', 1), ('appetite', 168), ('appetitie', 5), ('appetitte', 1), ('appettite', 1), ('appietite', 1), ('appitite', 1), ('dinner', 2), ('eat', 2), ('eaten', 1), ('eating', 19), ('food', 2), ('hunger', 1), ('hungry', 6), ('intake', 1), ('meal', 1), ('polyphadia', 1)]
menstrual = [('cycle', 9), ('cycles', 19), ('flow', 69), ('lmp', 142), ('lnmp', 1), ('menarche', 2), ('mennorhagia', 1), ('menorrhagea', 1), ('menorrhagia', 2), ('menses', 42), ('mensrtrual', 1), ('menstraul', 3), ('menstraution', 1), ('menstrual', 49), ('menstruation', 7), ('menstrucal', 1), ('menstural', 1), ('mentrual', 5), ('mentruation', 1), ('meses', 2), ('mesntruation', 1), ('mestrrual', 1), ('mestrual', 3), ('oligmenorrhoea', 1), ('oligomenorrhea', 1), ('pms', 4), ('polymenorrhea', 1), ('premenstrual', 1), ('rampons', 1), ('soaked', 1), ('tampom', 1), ('tampon', 20), ('tamponds', 1), ('tampons', 38), ('tampoon', 1), ('tampoons', 3)]
normal = [('abnormal', 1), ('nl', 2), ('normal', 42), ('normally', 3), ('regular', 56), ('regulr', 1), ('reularly', 1), ('typically', 2)]
pap_smear = [('pap', 26), ('papsmear', 1), ('smear', 17), ('smears', 1)]
iud = [('device', 3), ('intrauterine', 3), ('iud', 50), ('iuds', 1), ('uid', 1)]
dryness = [('dry', 1), ('dryness', 64), ('dyspareunia', 1), ('lubricant', 6), ('lubrication', 3)]
irregular = [('abnormal', 7), ('abnormalities', 2), ('inconsistent', 1), ('iregular', 1), ('irregualr', 1), ('irregular', 174), ('irregularities', 4), ('irregularity', 4), ('irregularly', 1), ('irregulites', 1), ('irreuglar', 2), ('irrgeular', 1), ('irrgualr', 1), ('irrgular', 1), ('random', 1), ('sometimes', 15), ('sometimjes', 1), ('unpredicatable', 1), ('unpredicatble', 1), ('unpredictable', 29), ('unpredictably', 1), ('unpredictible', 1), ('unpredictrable', 1), ('upredictable', 1), ('variable', 9), ('varies', 4), ('various', 3), ('vary', 2), ('varying', 1)]
periods = [('frequency', 4), ('frequent', 2), ('pattern', 19), ('patterns', 1), ('period', 60), ('periods', 253), ('perioeds', 1), ('perioids', 1), ('perior', 1), ('periords', 1), ('perios', 1), ('peroids', 3), ('preiods', 1), ('range', 14), ('ranged', 1), ('ranges', 7), ('ranging', 10)]
nausea = [('nasea', 1), ('nasua', 1), ('nasuea', 1), ('naucious', 1), ('nausa', 3), ('nause', 6), ('nausea', 207), ('nauseas', 4), ('nausease', 1), ('nauseated', 8), ('nauseous', 11), ('nauseu', 1), ('naussous', 1)]
vomit = [('vomit', 3), ('vomited', 12), ('vomiting', 77), ('vomitous', 1), ('vomits', 1), ('vomitted', 5), ('vomitting', 9), ('vomittng', 1), ('vomittus', 1), ('vomitus', 3), ('vomtiing', 1)]
flu = [('flu', 10)]
stress = [('depresion', 1), ('depress', 1), ('depressed', 6), ('depressin', 1), ('depression', 62), ('depressive', 3), ('dperession', 2), ('sad', 4), ('sadness', 3), ('stress', 36), ('stressed', 14), ('stresses', 1), ('stressful', 8), ('stressor', 6), ('stressors', 18), ('suffers', 1), ('suicidal', 13), ('suicidality', 2), ('suididial', 1), ('tressed', 1), ('unemployed', 15), ('unemployment', 3), ('worried', 1), ('worrying', 1)]
job_or_condo = [('apartment', 1), ('condo', 34), ('condominium', 1), ('job', 53)]
hot = [('diaphoresis', 5), ('hot', 128), ('hotflashes', 2), ('warm', 1), ('warmth', 2)]
cold = [('cold', 14)]
flash = [('flases', 1), ('flashes', 77), ('flshes', 1), ('hotflashes', 2)]
interval_or_duration = [('duration', 7), ('during', 1), ('interval', 3), ('intervals', 2), ('range', 13), ('ranged', 1), ('ranges', 7), ('ranging', 10)]
onset = [('began', 11), ('first', 9), ('started', 22), ('starting', 1), 'onset']
heavy = [('havier', 1), ('heavier', 13), ('heavily', 1), ('heaviness', 1), ('heavy', 87), ('heavyier', 1), ('hevier', 1)]
sweating = [('diaphoresis', 11), ('sweat', 12), ('sweatin', 1), ('sweatiness', 1), ('sweating', 60), ('sweatness', 1), ('sweats', 37), ('sweeting', 1), ('sweta', 1)]
night_or_morning = [('morning', 2), ('night', 57), ('nights', 1)]
sleep_or_falling = [('fall', 20), ('falling', 118), ('fallling', 1), ('falls', 2), ('fell', 1), ('slee', 1), ('sleep', 70), ('sleepin', 1), ('sleeping', 113), ('sleepng', 1), ('sleeps', 13)]
up = [('up', 26)]
peptic = [('pep', 1), ('peptic', 12), ('pud', 13)]
died = [('dead', 1), ('death', 18), ('died', 35), ('dies', 1), ('end', 1), ('killed', 2)]
discomfort = [('discomfort', 2)]
epigastric = [('eigastrium', 1), ('eipgastric', 1), ('epidrual', 1), ('epigartric', 1), ('epigasric', 1), ('epigastic', 1), ('epigastric', 76), ('epigastrium', 4), ('mid', 7), ('middle', 3), ('midepigastric', 3)]
area = [('area', 10)]
dark = [('black', 4), ('brown', 10), ('color', 3), ('colored', 3), ('colour', 1), ('dark', 31), ('darkened', 5), ('darkening', 8), ('darker', 31), ('darknmesing', 1), ('darkstools', 1), ('darlk', 1), ('draker', 1)]
stool = [('hematochezia', 2), ('hematoschezia', 1), ('hemochezia', 1), ('malena', 1), ('melana', 1), ('melena', 6), ('rectum', 3), ('stool', 41), ('stools', 51), ('tarry', 1)]
nsaid = [('acetaminophen', 1), ('ibuprofem', 1), ('ibuprofen', 64), ('ibuprogen', 1), ('ibuprophen', 1), ('medications', 1), ('meds', 1), ('moertin', 1), ('moltrin', 3), ('mortin', 2), ('mortrin', 2), ('motlin', 1), ('motrin', 88), ('motrins', 1), ('motron', 1), ('mottrin', 1), ('nsaid', 5), ('nsaids', 6), ('relief', 31), ('relieve', 3), ('relieved', 7), ('relievers', 1), ('relieving', 2), ('relived', 2), ('tyelenol', 1), ('tyenol', 1), ('tylemnol', 1), ('tylenol', 52), ('tylenon', 1), ('tylenonl', 1)]
burning_or_gnawing = [('buring', 2), ('burnig', 1), ('burning', 64), ('burninng', 1), ('burniung', 1), ('burnning', 1), ('gnawing', 28), ('gnowing', 1), ('knawing', 5), ('nagging', 2), ('nawing', 5), ('ngawing', 1)]
bloating = [('bloasting', 1), ('bloated', 14), ('bloating', 43), ('distenstion', 1), ('fullness', 5)]
eat = [('eating', 11), ('eats', 2), ('food', 4), ('intake', 1), ('meal', 2), ('meals', 13)]
feeling = [('feel', 7), ('feeling', 58), ('feelings', 10), ('feels', 36), ('feelslike', 2), ('sensation', 6), ('sense', 8), ('sesnations', 1)]
worse = [('aggrab', 1), ('aggrav', 2), ('aggravated', 6), ('aggravating', 1), ('aggraveted', 1), ('aggrevated', 1), ('exacerbated', 4), ('exacrebated', 1), ('progresion', 1), ('progressed', 2), ('progressing', 1), ('progressive', 7), ('progressively', 9), ('severe', 2), ('severity', 1), ('symptoms', 1), ('worse', 58), ('worseing', 1), ('worsen', 1), ('worsened', 9), ('worsening', 14), ('worsens', 8), ('worses', 1)]
alcohol = [('alcohol', 8), ('alcoholic', 3), ('beer', 7), ('beers', 36), ('beverage', 1), ('beverages', 2), ('drank', 1), ('drink', 2), ('drinks', 11), ('oz', 1)]
awake = [('awake', 1), ('awaken', 1), ('awakening', 11), ('awakenings', 5), ('awakens', 2), ('awakes', 1), ('awaking', 1), ('awakining', 1), ('awkaens', 1), ('awkening', 1), ('awoken', 1), ('wake', 4), ('wakes', 31), ('waking', 25)]
hematochezia = [('hematochezia', 2), ('hematoschezia', 1), ('hemochezia', 1)]
tums = [('allev', 2), ('antacid', 1), ('antacids', 4), ('tams', 1), ('thums', 1), ('toms', 2), ('tum', 1), ('tumes', 1), ('tums', 70)]
palpitations = [('beat', 1), ('beating', 2), ('palipations', 1), ('paliptations', 1), ('palpatations', 5), ('palpation', 1), ('palpations', 2), ('palpiations', 1), ('palpitaion', 5), ('palpitaions', 1), ('palpitation', 17), ('palpitations', 169), ('plapations', 1), ('plapitations', 1)]
anxious_or_nervous = [('anxiety', 31), ('anxious', 8), ('anxiousness', 2), ('nerviousness', 2), ('nervous', 29), ('nervouseness', 1), ('nervousenss', 2), ('nervousnes', 1), ('nervousness', 137), ('nervousnous', 1), ('neurvousness', 1), ('overwhelmed', 9)]
parents = [('parents', 6)]
elder = [('elderly', 14), ('older', 1)]
care = [('care', 43), ('caregiver', 1), ('carer', 1), ('cares', 3), ('caretaker', 3), ('caring', 6)]
home = ['home']
problem = [('problem', 1), ('problems', 1), ('trouble', 13), ('truble', 1)]
associated = [('asosciated', 2), ('assoc', 9), ('assocaited', 1), ('associated', 56), ('associates', 2)]
subjective = [('subjective', 42)]
emergency_department = [('department', 4), ('emergency', 6)] 
workup = [('workup', 12)]
result_description = [('bloodwork', 1), ('bmp', 7), ('bpm', 2), ('cardiac', 31), ('cbc', 35), ('chem', 1), ('cmp', 8), ('ecg', 22), ('ed', 44), ('ekg', 18), ('electrocardiogram', 1), ('electrolytes', 2), ('enzyme', 1), ('enzymes', 26), ('er', 14), ('markers', 2), ('metabolic', 19), ('tests', 1), ('tropinin', 2), ('troponins', 4), ('unremarkable', 6), ('unrevelaing', 2), ('wnl', 14)]
drug = [('does', 3), ('drug', 36), ('drugs', 44), ('durg', 1), ('ilicit', 2), ('illegeal', 1), ('illicit', 25), ('illiciti', 1), ('illicits', 1), ('illlicit', 1), ('illucit', 1), ('tobac', 1), ('toxic', 2)]
frequency = [('freq', 1), ('frequency', 21), ('frequent', 12), ('frequently', 2), ('freuqent', 1), ('infrequent', 1), ('infrequently', 2)]
doom = [('death', 1), ('die', 11), ('doom', 16), ('impending', 12)]
throat = [('thoat', 1), ('those', 1), ('thraot', 1), ('throat', 77)]
tightness = [('tighness', 1), ('tight', 5), ('tightenig', 1), ('tightening', 13), ('tightens', 1), ('tighting', 2), ('tightness', 23), ('toghtness', 1)]
clammy = [ ('calmy', 1), ('claminess', 2), ('clamminess', 4), ('clammy', 45), ('clammyness', 1), ('clamy', 9), ('clummy', 1)]
numbness = [('numb', 10), ('numbeness', 1), ('numbess', 1), ('numbing', 1), ('numbnes', 1), ('numbness', 46), ('numbs', 1), ('nymbess', 1), ('tingling', 8)]
finger_or_hand = [('finger', 14), ('fingers', 38), ('fingertips', 2), ('hand', 4), ('hands', 10)]
fatique = [('drain', 1), ('drained', 10), ('drowsiness', 1), ('fatiged', 1), ('fatigue', 35), ('fatigued', 22), ('fatigues', 1), ('fatiues', 1), ('reduced', 1), ('rested', 2), ('sleepiness', 1), ('sleepy', 1), ('somulance', 1), ('tire', 1), ('tired', 33), ('tiredness', 6)]
concentrating = [('concenrating', 1), ('concentrate', 2), ('concentrating', 7), ('concentration', 4), ('concetration', 1)]
fever = [('fever', 66), ('feverish', 6), ('fevers', 12), ('fevr', 1), ('hot', 2), ('temperatures', 1), ('touch', 1), ('warm', 51), ('warmth', 5)]
upper_respiratory = [('congested', 2), ('congestion', 19), ('congestions', 1), ('infection', 2), ('respiratory', 2), ('rhinnorhea', 1), ('rhinorhea', 1), ('rhinorrhea', 6), ('running', 1), ('runny', 14), ('ruuny', 1), ('secretions', 1), ('sinuses', 1), ('stuffed', 1), ('stuffines', 1), ('stuffiness', 5), ('stuffy', 44), ('tract', 1), ('upper', 2), ('uri', 5), ('urti', 1), ('viral', 1), ('whitish', 1), ('yellowish', 1)]
deep = [('deep', 78)]
pleuritic =  [('pleuretic', 1), ('pleuritic', 5)]
exercise = [('evercised', 1), ('excercise', 4), ('excercize', 1), ('excerise', 2), ('excersise', 1), ('excersize', 1), ('execise', 1), ('exercis', 1), ('exercise', 80), ('exercised', 5), ('exerxcise', 1)]
asthma = [('ashtma', 1), ('astham', 1), ('asthma', 103)]
rock_or_limbing =  [('carpenter', 2), ('climb', 3), ('climbed', 1), ('climber', 6), ('climbing', 14), ('climbings', 1), ('climbs', 8), ('gym', 1), ('lifting', 5), ('rock', 32), ('rockclimbing', 3)]
relief = [('acetaminophen', 1), ('alleviate', 3), ('alleviated', 8), ('alleviation', 2), ('ibuprofen', 58), ('ibuprogen', 1), ('ibuprophen', 1), ('relief', 44), ('relieve', 5), ('relieved', 14), ('relievers', 1), ('relieving', 2), ('relive', 1), ('relived', 3), ('tyelenol', 1), ('tyenol', 1), ('tylemnol', 1), ('tylenol', 52), ('tylenon', 1), ('tylenonl', 1)]
inhaler = [('albuteral', 1), ('albuterol', 56), ('inhaler', 32)]
sharp_or_stabbing = [('knife', 2), ('sharp', 79), ('stabbing', 15), ('stabby', 1)]
infertility = [('child', 2), ('children', 1), ('conceive', 9), ('conceived', 1), ('conceiving', 1), ('conception', 1), ('concieve', 2), ('inability', 2), ('incapable', 1), ('infertile', 3), ('infertility', 20), ('infertilliy', 1), ('pregant', 3), ('pregnant', 47), ('pregnany', 1), ('pregnat', 1), ('pregnent', 1), ('success', 3), ('successful', 1), ('sucess', 1), ('trial', 1), ('tried', 14), ('triend', 1), ('try', 2), ('trying', 6), ('unable', 18), ('unsuccesful', 1), ('unsuccessflly', 1), ('unsuccessful', 1), ('unsuccessfull', 1), ('unsuccessfully', 2), ('unsuccussfully', 1), 'able']
audio = [('audio', 1), ('auditory', 15), ('audiutory', 1), ('door', 5), ('hear', 3), ('heard', 17), ('hearing', 21), ('loud', 3), ('muffled', 5), ('music', 10), ('neigbor', 1), ('neighbor', 10), ('neighbors', 4), ('neightbor', 1), ('party', 19), ('things', 4), ('voices', 13)]
visual = [('saw', 13), ('seeing', 24), ('seen', 3), ('sees', 1), ('son', 43), ('vision', 1), ('visions', 2), ('visual', 19)]
hallucination = [('flashbacks', 1), ('halcuinations', 1), ('hallucination', 9), ('hallucinations', 32), ('hallusination', 1), ('hypnogogic', 1)]
tossing_or_turning = [('tossed', 1), ('tosses', 7), ('tossing', 15), ('turinin', 1), ('turned', 1), ('turning', 14), ('turns', 7)]
ambien = [('ambien', 1)]
taking = [('taking', 1)]
unsuccess = [('able', 1), ('fall', 3), ('unable', 3)]
sleep_medication = [('ambein', 1), ('ambian', 1), ('ambiant', 1), ('ambien', 48), ('ambiens', 1), ('ambient', 1), ('ampien', 1), ('medications', 1), ('pill', 1), ('pills', 1)]
help_or_nhelp = [('effect', 1), ('effective', 1), ('failed', 1), ('help', 16), ('helped', 12), ('helpful', 1), ('her', 3), ('imporovemnt', 1), ('improvement', 3), ('improves', 1), ('releif', 1), ('relief', 8), ('relieve', 1), ('relieved', 1), ('success', 1), ('wor', 1), ('work', 1)]
interest = [('activites', 1), ('activitie', 1), ('anhedonia', 14), ('enjoy', 1), ('enjoying', 1), ('inability', 1), ('interest', 54), ('interested', 2), ('interests', 2), ('intersest', 1), ('interst', 1), ('intrest', 2), ('lost', 15), ('uninterested', 2)]
ideations_or_thoughts = [('ideation', 29), ('ideations', 4), ('plan', 1), ('thoughts', 21)]
suicidal = [('homicidal', 1), ('hurting', 4), ('si', 7), ('sucidal', 1), ('sucidial', 1), ('suicida', 1), ('suicidadal', 1), ('suicidal', 41), ('suicidality', 7), ('suicide', 5), ('suidical', 1), ('suiidal', 1)]
myalgias_or_aches_or_achy = [('ache', 1), ('aches', 15), ('achiness', 1), ('achy', 8), ('bodyache', 1), ('ha', 7), ('hea', 1), ('heacahe', 1), ('headach', 1), ('headache', 47), ('headaches', 2), ('holocephalic', 1), ('holocranial', 2), ('hurting', 1), ('myalgia', 1), ('myalgias', 2), ('mylagias', 1)]
body_or_muscles = [('body', 12), ('muscle', 2)]
head = [('front', 1), ('head', 31)]
global_ = [('globa', 1), ('global', 2)]
neck = [('neck', 39)]
rash = [('rash', 6), ('rashes', 5)]
scratchy_or_viral_or_rhinorrhea = [('congestion', 2), ('cough', 1), ('discharge', 1), ('drainage', 1), ('flu', 1), ('itchy', 1), ('nasal', 3), ('nos', 1), ('nose', 19), ('respiratory', 1), ('rhinorrea', 1), ('rhinorrhea', 10), ('rihnorrhea', 1), ('running', 3), ('runny', 17), ('scrachy', 1), ('scratchy', 12), ('scrathy', 1), ('sore', 10), ('soreness', 1), ('sorethroat', 1), ('st', 1), ('thraot', 1), ('throat', 25), ('upper', 1), ('uri', 4)]
lives_an_apartment = [('apartment', 6), ('appartment', 1), ('apt', 1), ('home', 2), ('lives', 22), ('living', 1), ('room', 2), ('roomate', 10), ('roommate', 10)]
vaccine_or_meningococcal = [('meningitis', 1), ('meningococcal', 1), ('vaccinations', 1), 'shot']
migraines = [('magraine', 2), ('migaraines', 2), ('migarine', 2), ('migraine', 44), ('migrained', 2), ('migraines', 83), ('migrains', 6), ('migrane', 6), ('migranes', 7), ('migranies', 2), ('migrine', 2)] 
photophobia_or_light = [('bright', 10), ('briht', 1), ('eye', 1), ('eyes', 4), ('light', 24), ('lights', 11), ('photobia', 1), ('photohobia', 1), ('photophobi', 1), ('photophobia', 45), ('photophonia', 1), ('photosensitivity', 3)]
contact = [('contact', 3), ('contacts', 36), ('contcts', 1)]
illness = [('ill', 4), ('illness', 2)]

caffeine_2 = [('cae', 1),
('caeffienated', 1),
('caeffine', 1),
('caes', 1),
('caf', 1),
('cafaiene', 1),
('cafe', 3),
('cafeein', 1),
('cafeeinated', 1),
('cafeeine', 3),
('cafein', 1),
('cafeinated', 1),
('cafeine', 10),
('cafeteria', 19),
('caff', 2),
('caffaein', 1),
('caffaeine', 1),
('caffaine', 3),
('caffainex', 1),
('caffe', 6),
('caffee', 5),
('caffeein', 1),
('caffeeine', 2),
('caffees', 1),
('caffei', 1),
('caffeice', 1),
('caffeicne', 1),
('caffeien', 2),
('caffeienated', 2),
('caffeiene', 9),
('caffeife', 1),
('caffein', 45),
('caffeina', 1),
('caffeinated', 90),
('caffeind', 1),
('caffeine', 1687),
('caffeined', 1),
('caffeines', 2),
('caffeing', 4),
('caffeinge', 1),
('caffeinie', 2),
('caffeinr', 1),
('caffeins', 2),
('caffeint', 1),
('caffeintated', 1),
('caffenaited', 1),
('caffenated', 5),
('caffene', 6),
('caffeniated', 1),
('caffenie', 3),
('caffenine', 5),
('caffenne', 1),
('cafferine', 1),
('caffeteria', 2),
('cafffeine', 2),
('cafffiene', 1),
('caffie', 1),
('caffieen', 1),
('caffieine', 6),
('caffien', 14),
('caffienated', 18),
('caffience', 6),
('caffiene', 275),
('caffieneated', 1),
('caffienece', 1),
('caffinated', 18),
('caffine', 90),
('caffineate', 1),
('caffineated', 1),
('caffinee', 1),
('cafiene', 4),
('cafinated', 3),
('cafine', 3),  
('coeffee', 2),
('cof', 1),
('cofee', 33),
('cofeein', 1),
('coffaneited', 1),
('coffe', 229),
('coffea', 1),
('coffee', 3232),
('coffeeday', 1),
('coffeee', 19),
('coffeeground', 1),
('coffeer', 2),
('coffees', 65),
('coffeex', 1),
('coffeinated', 2),
('coffeine', 13),
('coffen', 1),
('coffer', 1),
('coffes', 6),
('coffess', 1),
('cofffe', 3),
('cofffee', 5),
('cofffer', 1),
('coffie', 5),
('coffiene', 1),
('coffies', 1),
('coffinated', 1), 
('ceffeine', 1),
('decaf', 12),
('decaff', 1),
('decaffeinated', 2),
('decaffeine', 1),
('decaffenated', 1),
('decaffinated', 1), 
('redb', 1),
('redbul', 2),
('redbull', 72),
('redbulls', 24),
('ccup', 1),
('ccups', 2),     
('cuo', 1),
('cup', 880),
('cupa', 1),
('cupcoffee', 1),
('cupd', 1),
('cupds', 1),
('cupful', 2),
('cupfull', 2),
('cuple', 1),
('cupp', 2),
('cupper', 1),
('cupple', 2),
('cupps', 4),
('cuprs', 1),
('cups', 2334),
('cupsof', 1),
('cupt', 2),
]

heavy_2 = [
('dheavy', 1),
('heaty', 1),
('heave', 7),
('heaveier', 1),
('heaveir', 3),
('heaver', 3),
('heavery', 3),
('heavey', 14),
('heavieer', 1),
('heavier', 861),
('heaviers', 1),
('heavies', 5),
('heaviest', 189),
('heaviets', 1),
('heavily', 23),
('heavineses', 1),
('heaviness', 81),
('heaving', 11),
('heavior', 1),
('heavist', 1),
('heavliy', 1),
('heavly', 3),
('heavry', 1),
('heavt', 1),
('heavty', 1),
('heavuy', 1),
('heavvy', 1),
('heavy', 5748),
('heavyer', 1),
('heavyier', 1),
('heavylifting', 2),
('heavymennorrhea', 1),
('heavyness', 5),
('heay', 6),
('heayy', 1),
('ehavy', 1),
('withheavy', 1),
('veryheavy', 1),
('nonheavy', 1),
('mheavy', 1),
('jheavy', 1),
('hgeavy', 1),
('havy', 3),
('havavy', 1),
('haeavy', 2),
]

diarrhea_2 = [
('ddiarrhea', 1),
('diar', 1),
('diarddhea', 1),
('diarea', 1),
('diareah', 2),
('diareha', 4),
('diarerhea', 1),
('diarhea', 107),
('diarheaa', 3),
('diarheea', 1),
('diarher', 1),
('diarhhea', 37),
('diarhhoea', 2),
('diarhoea', 5),
('diarhrea', 2),
('diarhrhea', 1),
('diariea', 1),
('diaries', 1),
('diarr', 4),
('diarraha', 1),
('diarrahea', 3),
('diarrea', 33),
('diarreah', 23),
('diarreh', 1),
('diarreha', 28),
('diarrehal', 1),
('diarrehea', 7),
('diarrha', 19),
('diarrhae', 2),
('diarrhe', 15),
('diarrhea', 9189),
('diarrhead', 1),
('diarrheae', 2),
('diarrheah', 1),
('diarrheahead', 1),
('diarrheal', 2),
('diarrhear', 4),
('diarrheas', 4),
('diarrheea', 1),
('diarrheoa', 1),
('diarrhera', 2),
('diarrhes', 2),
('diarrhesa', 1),
('diarrhhea', 1),
('diarrhia', 6),
('diarrhjea', 1),
('diarrhoae', 1),
('diarrhoe', 4),
('diarrhoea', 117),
('diarrhoeao', 1),
('diarrhrea', 2),
('diarrreha', 1),
('diarrrhea', 9),
('diarrthea', 1),
('diary', 1),
('nodiarrhea', 1),
('nondiarrheal', 1),
('noticeddiarhea', 1),
('darrhea', 1),
('darrhiea', 1),
]

watery_or_dark = [
('water', 193),
('waterbash', 1),
('waterbrash', 13),
('waterbrush', 1),
('watering', 21),
('waterny', 1),
('waterry', 3),
('watery', 381),
('waterying', 1),
('watrey', 1),
('wattery', 2),
('dark', 3450),
('darkbrown', 1),
('darkcolored', 1),
('darke', 4),
('darked', 6),
('darkeer', 2),
('darkeing', 4),
('darken', 8),
('darkend', 2),
('darkended', 1),
('darkened', 190),
('darkeneing', 3),
('darkenign', 3),
('darkenin', 1),
('darkening', 829),
('darkeningn', 1),
('darkeninig', 1),
('darkenng', 1),
('darkenning', 1),
('darker', 4257),
('darkerning', 1),
('darkers', 4),
('darkerx', 1),
('darkess', 1),
('darkiening', 1),
('darking', 24),
('darkining', 2),
('darkish', 4),
('darkned', 2),
('darkneing', 3),
('darknened', 1),
('darknening', 2),
('darkner', 1),
('darkness', 41),
('darkning', 1),
('darknmesing', 1),
('darks', 3),
('darkstools', 1),
('darlk', 1),
('darnk', 1),
('darnkening', 1),
('black', 651),
('blacked', 5),
('blackened', 3),
('blacker', 2),
('blacking', 3),
('blackish', 15),
('blackouts', 4),
('blacks', 1),
('broun', 1),
('brouwn', 1),
('brow', 1),
('browin', 1),
('browinish', 1),
('browm', 2),
('browmn', 1),
('brown', 1212),
('browncolor', 1),
('browner', 2),
('browning', 4),
('brownish', 24),
('brownpatch', 1),
('browns', 1),
('dbrown', 1),
('lightbrown', 2),
]

associated_2 = [
('asso', 37),
('assoaciated', 4),
('assoaciatio', 1),
('assoacited', 1),
('assoaisted', 1),
('assoc', 301),
('assocaiated', 11),
('assocaiation', 1),
('assocaitated', 1),
('assocaite', 2),
('assocaited', 187),
('assocaition', 6),
('assocaiton', 1),
('assocated', 27),
('assocatiated', 1),
('assocatied', 4),
('assocation', 15),
('assocations', 1),
('assocauted', 1),
('assocaution', 1),
('assocayed', 1),
('assoccaits', 1),
('assocciated', 2),
('assocd', 1),
('assocdiated', 1),
('associaated', 2),
('associaded', 1),
('associaed', 3),
('associaion', 1),
('associalted', 1),
('associarted', 2),
('associasted', 1),
('associatation', 3),
('associatd', 11),
('associate', 226),
('associatec', 1),
('associated', 13039),
('associatedc', 1),
('associatedd', 1),
('associatede', 1),
('associatedly', 1),
('associateds', 1),
('associatedw', 1),
('associateed', 5),
('associates', 66),
('associatesd', 1),
('associatetion', 1),
('associatiated', 1),
('associatied', 2),
('associatin', 1),
('associating', 18),
('associatioin', 1),
('association', 815),
('associationg', 1),
('associations', 38),
('associatted', 4),
('associattion', 1),
('associatwed', 1),
('associatyed', 1),
('associcated', 1),
('associeated', 1),
('associeted', 3),
('associiated', 1),
('associoated', 2),
('associsted', 1),
('associtaed', 6),
('associtated', 4),
('associted', 42),
('associtedw', 1),
('associuated', 1),
('assocoated', 2),
('assoctiated', 1),
('assoction', 1),
('assocuated', 1),
('assoiacted', 1),
('assoiated', 10),
('assoicate', 2),
('assoicated', 98),
('assoication', 4),
('assoications', 1),
('assoiciated', 1),
('assoiciation', 1),
('assoicitaed', 1),
('assoicite', 1),
('assoictaed', 1),
('assoicted', 3),
('assoociated', 3),
('assos', 3),
('assosciate', 2),
('assosciated', 15),
('assosiate', 1),
('assosiated', 17),
('assosicated', 1),
('assosited', 1),
('assosociated', 1),
('assossiated', 3),
('assotian', 1),
('assotiate', 1),
('assoviated', 1),
('asssistant', 2),
('asssoci', 1),
('asssociate', 1),
('asssociated', 31),
('asssociates', 1),
('asssociation', 1),
('asssocitaed', 1),
('asssoiated', 1),
('asst', 6),
('asstd', 1),
('episodeaccompanied', 1),
('aaccompanied', 1),
('accoacions', 1),
('accoasion', 1),
('accociated', 4),
('accocmpanied', 1),
('accomanied', 3),
('accomaonied', 1),
('accomapanied', 2),
('accomapined', 1),
('accomapnied', 1),
('accomodated', 1),
('accomodation', 1),
('accomopanids', 1),
('accomopanied', 1),
('accomp', 4),
('accompagned', 1),
('accompagnied', 1),
('accompained', 34),
('accompainied', 4),
('accompalied', 1),
('accompanbied', 1),
('accompaned', 1),
('accompanie', 2),
('accompanied', 1218),
('accompaniedd', 1),
('accompanies', 27),
('accompanined', 2),
('accompaning', 3),
('accompannied', 3),
('accompany', 8),
('accompanyied', 1),
('accompanyies', 1),
('accompanying', 104),
('accompaying', 2),
('accompied', 1),
('accompinaed', 1),
('accompined', 4),
('accomplaned', 1),
('accomplanied', 2),
('accomplish', 7),
('accomplishing', 1),
('accompnaied', 3),
('accompnaying', 1),
('accompnied', 4),
('accompnies', 1),
('accompoanied', 2),
('accoompanied', 1),
('accopanied', 5),
('accopanying', 1),
('accopmanied', 1),
('accopmanies', 1),
('accopnying', 1),
('accumpanied', 1),
('incl', 7),
('incld', 1),
('incle', 1),
('incloudes', 1),
('inclouding', 1),
('inclu', 1),
('inclucing', 2),
('includde', 1),
('include', 928),
('includea', 1),
('included', 166),
('includes', 194),
('includiing', 1),
('includin', 2),
('includind', 1),
('including', 1372),
('includinhg', 1),
('includng', 1),
('includnig', 1),
('includying', 1),
('inclue', 2),
('inclues', 1),
('incluidng', 2),
('inclujde', 1),
('inclulding', 1),
('inclusing', 1),
('inclusive', 1),
]

feel = [
('notfeel', 1),
('afeeling', 2),
('deniesfeeling', 1),
('deniesfeelings', 1),
('dfeel', 1),
('fee', 8),
('feeel', 1),
('feeelin', 1),
('feeeling', 21),
('feeelings', 2),
('feeels', 5),
('feeing', 10),
('feeings', 2),
('feekibg', 1),
('feeks', 1),
('feel', 2060),
('feela', 2),
('feeld', 1),
('feeled', 1),
('feeleing', 1),
('feeles', 2),
('feelgs', 1),
('feelhopeless', 1),
('feeli', 2),
('feeliag', 1),
('feelign', 11),
('feeligns', 5),
('feeligs', 1),
('feeliing', 3),
('feelikng', 1),
('feelimg', 1),
('feelimgs', 1),
('feelimng', 1),
('feelin', 15),
('feelind', 1),
('feelinf', 1),
('feeling', 11104),
('feelingg', 1),
('feelinglike', 1),
('feelings', 2301),
('feelingsd', 1),
('feelingsof', 1),
('feelinh', 2),
('feelinig', 1),
('feelinngs', 2),
('feelins', 8),
('feelint', 1),
('feelinvg', 1),
('feeliong', 1),
('feelis', 1),
('feelkingmore', 1),
('feelling', 7),
('feellings', 1),
('feells', 1),
('feelng', 5),
('feelngs', 2),
('feelnig', 5),
('feels', 10961),
('feelsd', 1),
('feelsing', 2),
('feelsings', 2),
('feelslike', 1),
('feelt', 1),
('feelts', 2),
('feely', 2),
('feelying', 1),
('ffeels', 1),
('ffels', 2),
('gfeeling', 1),
]

iregullar = [
('forirregular', 1),
('iiregular', 6),
('iirregular', 1),
('ireegualr', 1),
('ireegular', 10),
('ireggular', 4),
('iregular', 14),
('iregularities', 2),
('iregularity', 1),
('irergular', 2),
('irrecgular', 1),
('irrecular', 2),
('irreegular', 2),
('irrefular', 1),
('irreg', 10),
('irregardless', 1),
('irregaular', 1),
('irregeular', 1),
('irregilar', 2),
('irreglar', 4),
('irreglaritis', 1),
('irregluar', 4),
('irregual', 3),
('irregualar', 5),
('irregualarity', 1),
('irregualarly', 1),
('irregualr', 34),
('irregualrites', 1),
('irregualrities', 3),
('irregualry', 1),
('irreguar', 7),
('irreguarl', 1),
('irreguarly', 1),
('irreguklairties', 1),
('irreguklar', 1),
('irregula', 4),
('irregulairities', 1),
('irregulairity', 1),
('irregulairties', 1),
('irregular', 5149),
('irregularaties', 1),
('irregulare', 1),
('irregulargly', 1),
('irregularies', 1),
('irregularily', 1),
('irregularit', 1),
('irregularites', 3),
('irregularitie', 1),
('irregularities', 223),
('irregularitues', 1),
('irregularity', 183),
('irregulariy', 2),
('irregularlity', 3),
('irregularly', 54),
('irregularmentrual', 1),
('irregularperiods', 1),
('irregularrly', 1),
('irregulars', 1),
('irregularties', 4),
('irregulary', 4),
('irregulat', 3),
('irregulaties', 3),
('irregulation', 1),
('irregulatr', 2),
('irreguler', 2),
('irregulites', 1),
('irregullar', 1),
('irregulr', 2),
('irregural', 2),
('irreguylar', 1),
('irregylar', 1),
('irreh', 1),
('irreigular', 1),
('irreuglar', 2),
('irreular', 4),
('irrewgular', 1),
('irrgelar', 1),
('irrgellar', 1),
('irrgerular', 1),
('irrgeular', 15),
('irrgeularities', 1),
('irrgualr', 2),
('irrgular', 29),
('irrgularly', 1),
('irribular', 1),
('irrigegular', 1),
('irrigeular', 1),
('irriggulereties', 1),
('irrigualr', 1),
('irrigular', 13),
('irrlegular', 1),
('irrlevant', 1),
('irrregular', 5),
('irrugalr', 1),
('irruglar', 1),
('irrugralrites', 1),
('irrugular', 1),
('irrwgeular', 1),
('abmormal', 1),
('abmorn', 1),
('abn', 24),
('abnd', 3),
('abnl', 27),
('abnml', 8),
('abnnormal', 2),
('abnodmal', 1),
('abnoemla', 1),
('abnomal', 5),
('abnomalities', 3),
('abnomral', 7),
('abnomralities', 1),
('abnomrla', 1),
('abnomrmalities', 1),
('abnoormal', 1),
('abnoral', 2),
('abnoraml', 1),
('abnoramll', 1),
('abnorm', 1),
('abnormailties', 1),
('abnormaklities', 1),
('abnormal', 1561),
('abnormaliies', 1),
('abnormaliites', 1),
('abnormalites', 6),
('abnormalities', 199),
('abnormalitis', 1),
('abnormalitites', 2),
('abnormalitiy', 1),
('abnormalituiies', 1),
('abnormality', 25),
('abnormall', 1),
('abnormally', 55),
('abnormals', 5),
('abnormalties', 2),
('abnormaly', 2),
('abnorml', 1),
('abnormla', 2),
('abnormlaity', 1),
('abnormlities', 1),
('abnormltities', 1),
('abnormnality', 1),
('abnornal', 1),
('abnornmal', 1),
('abnrl', 1),
('abnrmal', 1),
('abnroal', 1),
('abnromal', 4),
('abnromalities', 1),
('abnromality', 1),
]

adderall_2 = [
('aderal', 50),
('aderall', 131),
('aderals', 3),
('aderel', 4),
('aderil', 1),
('adernal', 1),
('aderol', 108),
('aderole', 2),
('aderoll', 8),
('aderolol', 1),
('aderols', 1),
('aderral', 65),
('aderrall', 21),
('aderrol', 3),
('aderroll', 2),
('adheral', 5),
('adherall', 1),
('adherol', 2),
('addyrall', 1),
('addril', 2),
('addrolls', 2),
('addrrel', 2),
('addral', 1),
('addrall', 5),
('addreal', 3),
('addreall', 7),
('addrell', 10),
('addrella', 1),
('addrelle', 1),
('addreral', 1),
('addira', 2),
('addirall', 2),
('addheral', 2),
('adder', 1),
('adderaal', 3),
('adderal', 406),
('adderall', 1849),
('adderalla', 1),
('adderalll', 2),
('adderalls', 13),
('adderallto', 1),
('adderally', 3),
('adderals', 3),
('addereall', 1),
('adderell', 2),
('adderil', 1),
('adderol', 164),
('adderoll', 38),
('adderral', 115),
('adderrall', 81),
('adderrol', 9),
('adderroll', 1),
('adderrral', 1),
('addearall', 2),
('addarol', 1),
('addarral', 1),
('addderal', 1),
('addderall', 1),
('adarol', 3),
('adaral', 1),
('adarall', 1),
('adarell', 2),
('adaril', 2),
('dsadderall', 1),
('usesadderall', 1),
('alderal', 7),
('anderal', 2),
('amphatamine', 2),
('amphatamnine', 1),
('ampheatmine', 1),
('amphentamine', 1),
('amphetaine', 1),
('amphetamine', 53),
('amphetamines', 36),
('amphetimine', 1),
('amphitamine', 2),
('anfetamine', 1),
]

abdominal_2 = [
('andabdominal', 1),
('gnawingabdominal', 1),
('midabdominal', 1),
('sweatsabdominal', 1),
('aabdomen', 1),
('abdo', 63),
('abdoemal', 1),
('abdoiminal', 1),
('abdoinal', 10),
('abdoinla', 1),
('abdom', 4),
('abdomainal', 1),
('abdomainl', 5),
('abdomal', 2),
('abdoman', 1),
('abdome', 2),
('abdomeinal', 1),
('abdomeinlalpain', 1),
('abdomen', 545),
('abdomenal', 14),
('abdomenial', 1),
('abdoment', 1),
('abdomial', 13),
('abdomiala', 1),
('abdomian', 2),
('abdomianal', 1),
('abdomianl', 10),
('abdomin', 8),
('abdomina', 9),
('abdominak', 3),
('abdominal', 6909),
('abdominalk', 1),
('abdominalpain', 2),
('abdomincal', 1),
('abdominial', 2),
('abdominl', 2),
('abdominla', 8),
('abdominmal', 1),
('abdominnal', 1),
('abdomminal', 1),
('abdommninla', 1),
('abdomnal', 11),
('abdomne', 1),
('abdomnial', 8),
('abdomninal', 3),
('abdomoinal', 1),
('abdomonal', 2),
('abdomonial', 1),
('abdonial', 1),
('abdonimal', 1),
('abdoninal', 2),
('abdonminal', 2),
('abdoominal', 1),
('abdopain', 2),
('abdopminal', 1),
('andominal', 5),
('badominal', 1),
('baominal', 1),
('obdominal', 1),
('sbominal', 1),
('ubdominal', 1),
('abadominal', 1),
('abddominal', 3),
('abddominal', 3),
('abdl', 2),
('abdminal', 10),
('abdmoen', 2),
('abdmoinal', 4),
('abdmonial', 7),
('abdmoninal', 5),
('abodimnal', 1),
('abodinal', 1),
('abodmen', 3),
('abodminal', 24),
('abodmine', 1),
('abodominal', 5),
('abomdinal', 11),
('abomdminal', 1),
('abomen', 1),
('abomidnal', 1),
('abominal', 26),
('abomindal', 1),
('abominla', 1),
('abomnial', 1),
('absdominal', 1),
('absominal', 4),
('adbominal', 9),
('addominal', 1),
('adnominal', 1),
('adominal', 18),
('andabdominal', 1),
]

migraines_2 = [
('mig', 1),
('migain', 1),
('migaine', 2),
('migaines', 1),
('migaraines', 1),
('migarine', 4),
('migarines', 1),
('mighranes', 1),
('migr', 1),
('migraain', 1),
('migrai', 1),
('migraie', 1),
('migraies', 1),
('migraiine', 1),
('migraime', 1),
('migrain', 71),
('migraine', 1384),
('migrained', 3),
('migraineetoh', 1),
('migraines', 2943),
('migraineur', 1),
('migrainine', 1),
('migrainnes', 1),
('migrainous', 1),
('migrainr', 1),
('migrains', 69),
('migrainse', 1),
('migrainws', 1),
('migram', 1),
('migrame', 1),
('migrames', 1),
('migrane', 104),
('migranes', 174),
('migranges', 1),
('migranies', 2),
('migranine', 2),
('migranines', 2),
('migranins', 1),
('migrannes', 1),
('migrans', 4),
('migrein', 1),
('migriane', 1),
('migrianes', 6),
('migrine', 4),
('migrines', 1),
('miigraine', 1),
('mmigraines', 1),
('nonmig', 1),
('nonmigratory', 3),
('grain', 1),
('graine', 1),
('graines', 1),
('grains', 2),
('igraines', 2),
('igrains', 1),
('magraine', 2),
('maigrain', 1),
('maigraine', 2),
('megraine', 1),
('mgiraines', 1),
('mgraines', 1),
('miagraine', 1),
('miagraines', 2),
('miagrene', 1),
('miagrine', 1),
('mingraine', 1),
('miograin', 1),
('mirgaine', 1),
('mirgaines', 3),
('mirgrain', 1),
('mirgraine', 1),
('mirgraines', 8),
('miugraine', 1),
('mmigraines', 1),
('mograine', 3),
('mograines', 1),
('mugraine', 1),
('mygraine', 1),
('nigraine', 1),
('nigraines', 1),
('nigrans', 5),
]

thyroid_2 = [
('heperthyroid', 1),
('herthyroidism', 1),
('hjypothyroidism', 1),
('hyopothyroidism', 1),
('hyperparathyroidism', 1),
('hypertheyroidism', 1),
('hyperthiroidism', 1),
('hyperthroid', 1),
('hyperthroidism', 1),
('hyperthryoidism', 1),
('hyperthydism', 2),
('hyperthyoidisim', 1),
('hyperthyoidism', 1),
('hyperthyoridism', 1),
('hyperthyri', 1),
('hyperthyrodism', 3),
('hyperthyroid', 32),
('hyperthyroidism', 67),
('hypertyhroidism', 1),
('hypertyroidism', 1),
('hypethyriodism', 1),
('hypothryoid', 1),
('hypothryoidism', 1),
('hypothyoid', 1),
('hypothyoidisim', 1),
('hypothyriod', 1),
('hypothyrodism', 1),
('hypothyroid', 16),
('hypothyroidism', 27),
('hypthyroidism', 1),
('hyyperthyroidism', 1),
('parathyroid', 1),
('thyoid', 11),
('thyorid', 3),
('thyoroid', 1),
('thyrid', 4),
('thyriod', 10),
('thyrod', 3),
('thyroid', 2351),
('thyroidism', 3),
('thyroids', 2),
('thyroild', 1),
('thyroud', 1),
('yperthyroidism', 1),
('athyroid', 1),
('euthyroid', 1),
('hypertheyroidism', 1),
('hyperthiroidism', 1),
('hyperthroid', 1),
('hyperthroidism', 1),
('hyperthryoidism', 1),
('hyperthydism', 2),
('hyperthyoidisim', 1),
('hyperthyoidism', 1),
('hyperthyoridism', 1),
('hyperthyri', 1),
('hyperthyrodism', 3),
('hypertyhroidism', 1),
('hypertyroidism', 1),
('throid', 6),
('thryroid', 5),
('thryoid', 54),
('thryroid', 5),
('thryroides', 1),
('thryrois', 1),
('thuyroid', 1),
('thyoid', 11),
('thyorid', 3),
('thyoroid', 1),
('thyrid', 4),
('thyriod', 10),
('thyrod', 3),
('thyroid', 2351),
('thyroidism', 3),
('thyroids', 2),
('thyroild', 1),
('thyroimegaly', 1),
('thyromegally', 1),
('thyromegaly', 3),
('thyroud', 1),
('tyhroid', 1),
('tyreoid', 1),
('tyroid', 9),
('synthroid', 1),
('thiroid', 1),
]

epigastric_2 = [
('discomfortepigastric', 1),
('eidepigastric', 1),
('eigastric', 4),
('eigastrium', 1),
('epifastric', 1),
('epig', 3),
('epigaastric', 1),
('epigarstric', 1),
('epigartric', 2),
('epigasatrum', 1),
('epigasdtric', 1),
('epigasgric', 1),
('epigasric', 7),
('epigasrica', 1),
('epigasrrum', 1),
('epigasrtic', 6),
('epigasrtric', 1),
('epigasstric', 2),
('epigastalgia', 1),
('epigasteric', 8),
('epigasterium', 1),
('epigastgric', 1),
('epigastic', 117),
('epigastirc', 18),
('epigastiric', 2),
('epigastium', 3),
('epigastoric', 2),
('epigastr', 4),
('epigastral', 2),
('epigastralgia', 9),
('epigastrc', 2),
('epigastrci', 1),
('epigastreic', 1),
('epigastri', 6),
('epigastrial', 2),
('epigastrib', 1),
('epigastric', 7815),
('epigastrically', 4),
('epigastricly', 1),
('epigastricpain', 2),
('epigastrics', 1),
('epigastrict', 1),
('epigastrig', 1),
('epigastrim', 2),
('epigastrin', 2),
('epigastring', 1),
('epigastrio', 2),
('epigastrit', 1),
('epigastriuc', 1),
('epigastriuic', 1),
('epigastrium', 412),
('epigastriun', 1),
('epigastrtic', 1),
('epigastruim', 2),
('epigastrum', 119),
('epigastrus', 1),
('epigasttic', 1),
('epigasttric', 1),
('epigatric', 99),
('epigatricpain', 1),
('epigatrid', 1),
('epigatrium', 6),
('epigatrum', 2),
('epigatsic', 2),
('epigaxtric', 1),
('epigestric', 1),
('epiggastric', 2),
('epigigastric', 1),
('epigrastic', 5),
('epigrastirc', 1),
('epigrastric', 25),
('epigrastrium', 4),
('epigrastrum', 2),
('epigratric', 1),
('epigsastric', 2),
('epigsatric', 4),
('epigstaric', 1),
('epigstric', 24),
('epigstrium', 1),
('epipgastric', 2),
('episgastric', 10),
('episgastru', 1),
('episgatric', 2),
('epistaxis', 14),
('epistric', 1),
('epiugastric', 1),
('epiugestrium', 1),
('epogastric', 2),
('epoigastric', 1),
('eppigastric', 1),
('eppigastrig', 1),
('eppigatifc', 1),
('epugastic', 1),
('epuigastric', 1),
('epygastric', 1),
('inepigastric', 2),
('inepigastrium', 1),
('mdiepigastric', 1),
('midepiastric', 1),
('midepigartric', 1),
('midepigastic', 3),
('midepigastric', 191),
('midepigastrium', 2),
('midepigastrum', 2),
('midepigatric', 2),
('midepigrastic', 1),
('midepigrastric', 1),
('midgastric', 1),
('miedpigastric', 1),
('miepigastric', 1),
('ofepigastric', 1),
('upperepigastric', 1),
('egiastric', 1),
('egigastric', 1),
('egipastrium', 1),
('eidepigastric', 1),
('apigastric', 2),
('egiastric', 1),
('egigastric', 1),
('egipastrium', 1),
('eidepigastric', 1),
('eigastric', 4),
('eigastrium', 1),
('eipgastric', 3),
('eoigastrium', 1),
('epegastric', 1),
('epgastic', 2),
('epgastirc', 1),
('epgastric', 6),
('epgatsric', 1),
('epgiastric', 1),
('epgigastric', 1),
('epiagastic', 1),
('epiagrastric', 1),
('epiagstiric', 1),
('epiastric', 3),
('epicastric', 1),
('epidgastric', 2),
('epiugastric', 1),
('epiugestrium', 1),
('epogastric', 2),
('epoigastric', 1),
('eppigastric', 1),
('eppigastrig', 1),
('eppigatifc', 1),
('epugastic', 1),
('epuigastric', 1),
('epygastric', 1),
('inepigastric', 2),
('inepigastrium', 1),
('mdiepigastric', 1),
('midepiastric', 1),
('midepigartric', 1),
('midepigastic', 3),
('midepigastric', 191),
('midepigastrium', 2),
('midepigastrum', 2),
('midepigatric', 2),
('midepigrastic', 1),
('midepigrastric', 1),
('midgastric', 1),
('miedpigastric', 1),
('miepigastric', 1),
('ofepigastric', 1),
('peigastric', 4),
('peigastrim', 1),
('perigastric', 1),
('pigastric', 1),
('pigastrium', 1),
('respigastric', 1),
('rpigastric', 1),
('spigastric', 3),
('upperepigastric', 1),
('wpigastrium', 1),
]

tightness_2 = [
('tigentening', 1),
('tigethens', 1),
('tigh', 3),
('tighening', 7),
('tighenning', 1),
('tighens', 1),
('tighers', 1),
('tighet', 1),
('tigheting', 1),
('tighetingn', 1),
('tighetining', 1),
('tighgtening', 1),
('tighh', 1),
('tighhtening', 1),
('tighhtness', 1),
('tighing', 1),
('tighnes', 1),
('tighness', 47),
('tighnitess', 1),
('tighnness', 1),
('tighntess', 2),
('tighntness', 2),
('tight', 446),
('tightedness', 8),
('tighten', 20),
('tightened', 9),
('tighteness', 5),
('tightenig', 1),
('tightenin', 1),
('tightening', 885),
('tighteningduring', 1),
('tightenings', 1),
('tightenng', 3),
('tightennig', 1),
('tightenning', 1),
('tightenns', 1),
('tightens', 111),
('tightenss', 1),
('tightenting', 2),
('tighterning', 1),
('tightewndened', 1),
('tighting', 11),
('tightining', 2),
('tightly', 1),
('tightmess', 2),
('tightneess', 1),
('tightneing', 6),
('tightneness', 1),
('tightnening', 3),
('tightnes', 2),
('tightneses', 1),
('tightness', 2572),
('tightnessin', 1),
('tightnessm', 2),
('tightnesss', 2),
('tightnessw', 1),
('tightniess', 1),
('tightnig', 1),
('tightnigh', 1),
('tightning', 16),
('tightninig', 1),
('tightnness', 2),
('tightnses', 1),
('tightnss', 1),
('tigliness', 1),
('tignhtness', 1),
('tigntness', 1),
('tigtening', 2),
('tigtens', 1),
('tigthening', 2),
('tigthness', 3),
('tigthning', 1),
('tigthtened', 1),
('tigtness', 6),
('tigtning', 1),
('tihghtness', 1),
('titghtness', 1),
('lightlightness', 1),
('tghtness', 1),
('thgtness', 1),
('thighneess', 1),
('thighness', 4),
('thighten', 1),
('thighteness', 1),
('thightening', 1),
('thightens', 1),
('thightess', 1),
('thightining', 1),
('thightnes', 1),
('thightness', 29),
('thightning', 1),
('thigness', 1),
('thigtnes', 1),
('thigtness', 3),
('thingness', 1),
('thitness', 1),
('tighhtness', 1),
('tigh', 3),
('tighening', 7),
('tighenning', 1),
('tighens', 1),
('tighers', 1),
('tighet', 1),
('tigheting', 1),
('tighetingn', 1),
('tighetining', 1),
('tighgtening', 1),
('tighh', 1),
('tighhtening', 1),
('tighhtness', 1),
('tighnes', 1),
('tighness', 47),
('tighnitess', 1),
('tighnness', 1),
('tighntess', 2),
('tighntness', 2),
('tignhtness', 1),
('tigntness', 1),
('tigtening', 2),
('tigtens', 1),
('tigthening', 2),
('tigthness', 3),
('tigthning', 1),
('tigthtened', 1),
('tigtness', 6),
('tigtning', 1),
('tihghtness', 1),
('tihgt', 1),
('titghtness', 1),
('toghtness', 1),
('toightness', 1),
('trightness', 1),
]

heart_2 = [
('feelsheart', 1),
('gheart', 2),
('hearrt', 2),
('hears', 86),
('heart', 10827),
('hearth', 7),
('hearting', 5),
('heartno', 1),
('hearts', 8),
('heartt', 1),
('heartv', 1),
('herheart', 1),
('hgheart', 1),
('hheart', 1),
('eart', 1),
('earts', 1),
('ehart', 3),
('feart', 1),
('hart', 5),
('heaart', 1),
('heeart', 2),
('herart', 1),
('herat', 17),
('hreart', 2),
('hreat', 1),
('ohoeart', 1),
('yeart', 1),
('yearts', 1),
]

ache_2 = [
('aache', 1),
('ache', 1630),
('ached', 3),
('achediet', 1),
('achees', 1),
('acheid', 1),
('acheing', 4),
('acheness', 1),
('aches', 3538),
('achesand', 1),
('achessince', 1),
('achey', 176),
('acheyness', 5),
('achges', 1),
('achhes', 1),
('achhey', 1),
('ace', 14),
('acec', 1),
('acei', 10),
('aces', 1),
('acess', 1),
('acfhe', 1),
('acgive', 1),
('ach', 10),
('achanges', 1),
('achantosis', 1),
('ache', 1630),
('ached', 3),
('achediet', 1),
('acheduler', 1),
('achees', 1),
('acheid', 1),
('acheing', 4),
('acheive', 3),
('acheived', 2),
('acheivement', 1),
('acheness', 1),
('aches', 3538),
('achesand', 1),
('achessince', 1),
('achey', 176),
('acheyness', 5),
('achges', 1),
('achhes', 1),
('achhey', 1),
('achi', 1),
('achimg', 1),
('achiness', 71),
('aching', 597),
('achinging', 1),
('achinng', 1),
('achne', 1),
('achneges', 1),
('achng', 1),
('achnge', 1),
('acho', 1),
('achs', 2),
('achses', 1),
('achy', 915),
('achyand', 1),
('achying', 2),
('achyness', 10),
('acky', 1),
('adache', 1),
('aheadaches', 1),
('yeadaches', 1),
('theadache', 1),
('toothache', 3),
('stomachaches', 3),
('stomachache', 17),
('rheadache', 2),
('painheadache', 1),
('oheadache', 1),
('noheadaches', 1),
('aache', 1),
('ache', 1630),
('ached', 3),
('achediet', 1),
('acheduler', 1),
('achees', 1),
('acheid', 1),
('acheing', 4),
('acheness', 1),
('aches', 3538),
('achesand', 1),
('achessince', 1),
('achey', 176),
('acheyness', 5),
('adache', 1),
('aheadaches', 1),
('bachache', 2),
('bachaches', 4),
('backache', 73),
('backaches', 104),
('backeaches', 1),
('bakcache', 1),
('bodaches', 1),
('bodyache', 18),
('bodyaches', 32),
('bodybaches', 1),
('changeheadache', 1),
('chillsheadache', 1),
('constantache', 1),
('dullache', 5),
('eadache', 2),
('earache', 3),
('earachem', 1),
('earrache', 2),
('ehadache', 1),
('ehadaches', 3),
('eyeache', 1),
('hadache', 6),
('hadaches', 3),
('hadeache', 1),
('haeache', 1),
('haeadache', 1),
('haeadaches', 1),
('haedache', 7),
('haedaches', 1),
('hdeache', 1),
('heaache', 10),
('heaaches', 4),
('heaadache', 1),
('heacache', 5),
('heacaches', 1),
('heacdache', 1),
('heachache', 15),
('heachaches', 9),
('heache', 17),
('heacheaches', 1),
('heached', 1),
('heaches', 11),
('headaache', 4),
('headaaches', 3),
('headache', 13065),
('headachea', 2),
('headacheand', 1),
('headachechest', 1),
('headached', 15),
('headacheds', 1),
('headachee', 2),
('headachehas', 1),
('headachelike', 1),
('headachem', 2),
('headaches', 5600),
('headachese', 1),
('headachesince', 1),
('headachesm', 1),
('headachess', 1),
('headachex', 5),
('headcache', 1),
('headdache', 3),
('headeache', 7),
('headeaches', 3),
('headfache', 1),
('headfaches', 1),
('headhache', 3),
('headhaches', 1),
('headsaches', 1),
('heahache', 1),
('heahaches', 1),
('heaqdache', 1),
('hearaches', 1),
('heartache', 1),
('heasache', 1),
('heasaches', 1),
('hedache', 24),
('hedaches', 8),
('hedadache', 1),
('hedeache', 1),
('heeadache', 1),
('heradache', 1),
('hgeadache', 1),
('hheadache', 1),
('hjeadache', 1),
('hreadaches', 1),
('jheaches', 1),
('leadaches', 1),
('lightheadache', 1),
('manarache', 1),
('menache', 27),
('menarache', 13),
('menerache', 1),
('menrache', 4),
('merache', 4),
('mernache', 1),
('mildheadache', 1),
('mucleache', 1),
('muscleache', 3),
('muscleaches', 3),
('musleaches', 1),
('neckache', 1),
('nheadache', 1),
('noheadaches', 1),
('oheadache', 1),
('painheadache', 1),
('raches', 1),
('reseracher', 5),
('rheadache', 2),
('stomachache', 17),
('stomachaches', 3),
('theadache', 1),
('toothache', 3),
('yeadaches', 1),
]

appetite_2 = [
('aappetite', 2),
('appeetite', 1),
('appeite', 23),
('appeitei', 2),
('appeitie', 24),
('appeities', 1),
('appeitite', 57),
('appeitte', 2),
('appeittie', 1),
('appeitties', 1),
('appentit', 1),
('appenxix', 1),
('appepite', 8),
('appepities', 1),
('appepitite', 5),
('appeptie', 2),
('appeptir', 1),
('appeptite', 30),
('appeptitie', 3),
('apperite', 1),
('appertite', 3),
('appet', 3),
('appetatie', 1),
('appeteie', 1),
('appeteite', 10),
('appetes', 1),
('appetete', 1),
('appetetie', 2),
('appetetite', 3),
('appeti', 1),
('appetiate', 3),
('appetide', 13),
('appetie', 53),
('appetied', 1),
('appeties', 2),
('appetiet', 5),
('appetiete', 10),
('appetietie', 1),
('appetiie', 1),
('appetiite', 6),
('appetioite', 1),
('appetiote', 1),
('appetipe', 1),
('appetire', 4),
('appetirte', 1),
('appetit', 19),
('appetite', 12512),
('appetiteand', 2),
('appetitebu', 1),
('appetitechange', 1),
('appetited', 8),
('appetitedenies', 1),
('appetitehas', 1),
('appetiteie', 1),
('appetitel', 2),
('appetitem', 1),
('appetiteor', 1),
('appetitepast', 1),
('appetiters', 1),
('appetites', 16),
('appetitew', 1),
('appetitex', 4),
('appetitexmorning', 1),
('appetiti', 2),
('appetitie', 466),
('appetities', 4),
('appetitine', 1),
('appetitis', 3),
('appetitite', 29),
('appetitive', 2),
('appetitle', 2),
('appetitpe', 1),
('appetitre', 1),
('appetits', 2),
('appetitte', 19),
('appetiute', 1),
('appetoit', 1),
('appetrite', 1),
('appette', 3),
('appettie', 20),
('appettiet', 1),
('appettite', 55),
('appettitte', 1),
('appetuite', 1),
('appetyite', 1),
('appiete', 1),
('appietite', 2),
('appitatie', 1),
('appited', 1),
('appitete', 1),
('appitiate', 1),
('appitie', 7),
('appities', 1),
('appitit', 2),
('appitite', 70),
('appititeng', 1),
('appititie', 1),
('appittie', 1),
('appitties', 1),
('appittite', 2),
('apprtiete', 1),
('appt', 19),
('appteite', 5),
('apptetie', 2),
('apptetite', 5),
('apptiete', 1),
('apptite', 22),
('apptitite', 1),
('apptitte', 1),
('apptiye', 1),
('appts', 1),
('appttie', 1),
('decappetited', 1),
('decreasedxappetite', 1),
('eappetite', 1),
('inappetite', 2),
('increasedappetite', 2),
('nomappetite', 1),
('sappetitie', 1),
('aappetite', 2),
('aapetite', 1),
('apapetite', 1),
('apaptite', 1),
('apatite', 5),
('apatitie', 1),
('apeetite', 2),
('apeite', 1),
('apeitie', 2),
('apeitite', 2),
('apepeite', 1),
('apeptide', 1),
('apeptie', 1),
('apeptite', 18),
('apeptitite', 1),
('aper', 1),
('apetatie', 1),
('apeteite', 1),
('apetiet', 1),
('apetige', 1),
('apetiite', 1),
('apetit', 2),
('apetite', 526),
('apetites', 2),
('apetitet', 1),
('apetiti', 1),
('apetitie', 30),
('apetitite', 1),
('apetitre', 1),
('apetitte', 7),
('apetoite', 1),
('apette', 1),
('apettie', 4),
('apettite', 38),
('apettited', 1),
('apitite', 8),
('apopetite', 1),
('appaetite', 5),
('appatite', 16),
('appeetite', 1),
('appeite', 23),
('appeitei', 2),
('appeitie', 24),
('appeities', 1),
('appeitite', 57),
('appeitte', 2),
('appeittie', 1),
('appeitties', 1),
('ppetite', 2),
('petite', 2),
('hepptite', 1),
('epetite', 1),
('eappetite', 1),
('dappetite', 1),
('aptite', 2),
('aprtite', 1),
('apptitite', 1),
('apptite', 22),
('appietite', 2),
('appitite', 70),
('appititeng', 1),
('appittite', 2),
('applitite', 1),
('apppetite', 12),
('appreitite', 1),
('appretite', 6),
('apptetite', 5)
]

decrease_2 = [
('ddeceased', 1),
('ddecrased', 1),
('ddecreased', 1),
('dcerased', 1),
('dcerease', 1),
('dcreasd', 2),
('dcrease', 1),
('dcreased', 1),
('decerased', 6),
('decereased', 4),
('decereases', 1),
('decesaed', 1),
('decesased', 1),
('deceseaed', 1),
('deceseased', 2),
('deccreased', 3),
('deceaced', 1),
('deceaed', 1),
('decearsed', 1),
('deceas', 1),
('decease', 9),
('deceased', 488),
('deceasedd', 1),
('deceasedof', 1),
('deceases', 2),
('dececreasd', 1),
('dececreased', 1),
('decerased', 6),
('decereased', 4),
('decereases', 1),
('decesaed', 1),
('decesased', 1),
('deceseaed', 1),
('deceseased', 2),
('decesed', 1),
('decr', 34),
('decraese', 4),
('decraesed', 2),
('decraibes', 1),
('decrase', 3),
('decrasease', 1),
('decrased', 23),
('decre', 3),
('decreaase', 1),
('decreaased', 2),
('decread', 1),
('decreae', 2),
('decreaed', 27),
('decreaes', 4),
('decreaesd', 5),
('decreaesed', 1),
('decreas', 3),
('decreasd', 14),
('decrease', 1211),
('decreasead', 1),
('decreased', 7538),
('decreaseda', 1),
('decreaseds', 1),
('decreasedx', 1),
('decreasee', 1),
('decreases', 111),
('decreasesd', 1),
('decreasesed', 1),
('decreaseted', 1),
('decreasev', 1),
('decreasing', 90),
('decreasins', 1),
('decreasng', 1),
('decreassed', 1),
('decreast', 1),
('decreeased', 2),
('decrees', 1),
('decreibes', 1),
('decrereased', 1),
('decresae', 1),
('decresaed', 9),
('decresased', 1),
('decresasing', 1),
('decresde', 1),
('decrese', 15),
('decresead', 1),
('decreseased', 1),
('decresed', 56),
('decreses', 1),
('decresing', 1),
('decressed', 1),
('decrreased', 4),
('decrsead', 1),
('decrsease', 1),
('decrtease', 1),
('decseased', 1),
('dectease', 1),
('deecreased', 1),
('ordecreased', 1),
('dcerease', 1),
('dcrease', 1),
('dcreased', 1),
('hadincreased', 1),
('ecreased', 1),
('deecreased', 1),
('dercreased', 1),
('derease', 1),
('dereased', 5),
('descrease', 2),
('descreased', 4),
('devreased', 1),
('dicrease', 1),
('dicreased', 3),
('discreases', 1),
('dreased', 2),
('drecrease', 1),
('drecreased', 2),
('dcerease', 1),
('dcrease', 1),
('dcreased', 1),
('ddecreased', 1),
('deacrease', 1),
('deacreased', 4),
('deccreased', 3),
('dececreased', 1),
('decereased', 4),
('decereases', 1)
]

increase_2 = [
('eincreased', 1),
('hadincreased', 1),
('incr', 25),
('incraese', 1),
('incraesed', 7),
('incraeses', 2),
('incraesing', 1),
('incrase', 3),
('incrased', 17),
('incrasing', 2),
('incre', 1),
('increa', 1),
('increaased', 2),
('increaasing', 1),
('increaded', 1),
('increae', 3),
('increaed', 26),
('increaesd', 4),
('increaese', 1),
('increaesed', 1),
('increaing', 4),
('increaring', 1),
('increas', 5),
('increasae', 1),
('increasd', 7),
('increasde', 1),
('increasded', 1),
('increase', 1328),
('increasecd', 1),
('increased', 7881),
('increasedd', 3),
('increasedin', 1),
('increasedto', 1),
('increasee', 1),
('increaseed', 2),
('increaseedd', 1),
('increasein', 1),
('increaseing', 10),
('increaseingly', 1),
('increasenly', 1),
('increaserd', 1),
('increases', 364),
('increasesd', 2),
('increasid', 1),
('increasign', 1),
('increasily', 1),
('increasing', 1117),
('increasingluy', 1),
('increasingly', 158),
('increasinglyh', 1),
('increasinly', 1),
('increasng', 2),
('increasngly', 1),
('increasnig', 1),
('increasrf', 1),
('increass', 1),
('increasse', 2),
('increassed', 1),
('increast', 1),
('increasted', 1),
('increaswed', 1),
('increated', 1),
('incredbily', 1),
('incredible', 1),
('incredibly', 3),
('incredulous', 1),
('increeased', 3),
('increeasing', 1),
('increes', 1),
('increqued', 1),
('increreased', 1),
('incresaed', 5),
('incresaing', 1),
('incresase', 1),
('incresased', 2),
('incresasing', 1),
('increse', 8),
('incresease', 2),
('increseases', 1),
('incresed', 34),
('increses', 6),
('incresing', 3),
('incresingly', 1),
('incressed', 1),
('incressing', 1),
('increwasing', 1),
('incrrease', 1),
('incrreased', 1),
('incrseae', 1),
('incrwasing', 1),
('incting', 1),
('inctreased', 1),
('inctresed', 1),
('reportsincreased', 1),
('ncrease', 1),
('ncreased', 1),
('increreased', 1),
('incrrease', 1),
('incrreased', 1),
('inctreased', 1),
('inhcreased', 1),
('inrease', 1),
('inreased', 4),
('inrreased', 1),
('inscreased', 1),
('iuncrease', 1),
('hadincreased', 1),
('icnrease', 1),
('icnreased', 4),
('icrease', 4),
('icreased', 2),
('imcreased', 1),
('inccreases', 1)
]

In [ ]:
find = 'ache'

for count, row in words.iterrows():
    if row['word'].find('ache') != -1:
        print('(' + "'" +  row['word'] + "'" + ', ' + str(row['count']) + ")" + ',')

In [ ]:
num=702
display(features[features['feature_num']==num])
texts = train[(train['feature_num']==num)]
    
def word_counter(texts):
    words = []
    for index, row in texts.iterrows():
        annotation = eval(row['annotation'])
        for text in annotation:
            text = text_transform(text)
            text = text.split()
            words = words + text
    return Counter(words).most_common()

word_counter(texts)

In [ ]:
features

In [ ]:
rgroups_list = [('caffeine', caffeine + caffeine_2),
                ('heavy', heavy + heavy_2),
                ('diarrhea', diarrhea_2),
                ('watery', watery_or_dark),
                ('associated', associated_2),
                ('feeling', feel),
                ('iregullar', iregullar),
                ('adderall', adderall_2),
                ('abdominal', abdominal_2),
                ('migraines', migraines_2),
                ('thyroid', thyroid_2),
                ('epigastric', epigastric_2),
                ('tightness', tightness_2),
                ('heart', heart_2),
                ('ache', ache_2),
                ('appetite', appetite_2),
                ('decrease', decrease_2),
                ('increase', increase_2),
               ]

def aggregator(words):
    rwords = []
    for word in words:
        if type(word) == tuple:
            word = word[0]
        rwords.append(word)
    return rwords

rgroups_csv = pd.DataFrame(columns=['name', 'words'])
for rgroup in rgroups_list: 
    rgroups_csv = rgroups_csv.append({'name': rgroup[0], 'words': aggregator(rgroup[1])}, ignore_index=True)
display(rgroups_csv)

rgroups_csv.to_csv('rgroups_version2.csv', index=False)

In [ ]:
rgroups_csv.to_csv('rgroups_version1.csv', index=False)